# 1 load data and get prediction result

In [1]:
import demo_load_bay
import torch
engine = demo_load_bay.get_engine()
#create model

engine.model.load_state_dict(torch.load("garage/pems-bay_best_1.59.pth"))
#load param

yhat=demo_load_bay.get_time_T(engine)
#get predict result

engine get!


In [2]:
yhat.shape


torch.Size([10419, 325, 12])

# 2  read raw PEMS

In [3]:
import h5py
file='data/PEMS-BAY/pems-bay.h5'
 
f=h5py.File(file,'r')
f.keys()

<KeysViewHDF5 ['speed']>

In [4]:
f['speed']

<HDF5 group "/speed" (4 members)>

In [5]:
f['speed'].keys()

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values']>

## 2.1 time for PEMS-BAY

In [6]:
f['speed']['axis1'][:].shape,f['speed']['axis1'][:]

((52116,),
 array([1483228800000000000, 1483229100000000000, 1483229400000000000, ...,
        1498866300000000000, 1498866600000000000, 1498866900000000000]))

In [7]:
tst_time=f['speed']['axis1'][-10419:]
tst_time.shape

(10419,)

In [8]:
import pandas as pd
 
datetimes=pd.to_datetime(tst_time)
datetimes

DatetimeIndex(['2017-05-25 19:45:00', '2017-05-25 19:50:00',
               '2017-05-25 19:55:00', '2017-05-25 20:00:00',
               '2017-05-25 20:05:00', '2017-05-25 20:10:00',
               '2017-05-25 20:15:00', '2017-05-25 20:20:00',
               '2017-05-25 20:25:00', '2017-05-25 20:30:00',
               ...
               '2017-06-30 23:10:00', '2017-06-30 23:15:00',
               '2017-06-30 23:20:00', '2017-06-30 23:25:00',
               '2017-06-30 23:30:00', '2017-06-30 23:35:00',
               '2017-06-30 23:40:00', '2017-06-30 23:45:00',
               '2017-06-30 23:50:00', '2017-06-30 23:55:00'],
              dtype='datetime64[ns]', length=10419, freq=None)

## 2.2 raw value of PEMS-BAY

In [9]:
speed=f['speed']['block0_values'][:]

In [10]:
speed

array([[71.4, 67.8, 70.5, ..., 68.8, 71.1, 68. ],
       [71.6, 67.5, 70.6, ..., 68.4, 70.8, 67.4],
       [71.6, 67.6, 70.2, ..., 68.4, 70.5, 67.9],
       ...,
       [71.4, 66.9, 68.1, ..., 68.4, 71.6, 66.6],
       [72.2, 66.5, 68. , ..., 68.7, 71.6, 68.4],
       [71.5, 66.2, 68.4, ..., 68.7, 71.6, 68. ]])

In [11]:
mean,std=speed.mean(),speed.std()
mean,std

(62.619573607986965, 9.594371081990632)

# 3 PEMS-BAY location

In [12]:
import pandas as pd
df=pd.read_csv('data/PEMS-BAY/graph_sensor_locations_bay.csv',names=['station_id','lat','lon'])
df

,station_id,lat,lon
0,400001,37.364085,-121.901149
1,400017,37.253303,-121.945440
2,400030,37.359087,-121.906538
3,400040,37.294949,-121.873109
4,400045,37.363402,-121.902233
...,...,...,...
320,413845,37.422887,-121.925747
321,413877,37.321613,-121.899642
322,413878,37.324641,-121.888603
323,414284,37.323066,-121.896538


In [13]:
mean_lat=df['lat'].mean()
mean_lon=df['lon'].mean()
mean_lat,mean_lon

(37.344741424615385, -121.94008344615385)

In [14]:
import folium
 
 
m=folium.Map(location=(mean_lat,mean_lon),zoom_start=12)
 
for i in df.iterrows():
    #print(i)
    tmp_lat=i[1]['lat']
    tmp_lon=i[1]['lon']
    tmp_id=i[1]['station_id']
    #print(tmp_lat,tmp_lon)
    folium.Marker(location=(tmp_lat,tmp_lon),
                  icon=folium.Icon(icon='car',
                                   prefix='fa',
                                  color='green',
                                  icon_color='blue'),
                 popup=str(int(tmp_id))+' : '+str(tmp_lat)+' , '+str(tmp_lon)).add_to(m)
m

# 4 get a time slot's predicted speed

index 0 refers to 2017-05-25 19:45:00, index i refers to the ith 5min from index 0

In [15]:
num=int(input('input a number, from 0 to 10418'))


KeyboardInterrupt



In [16]:
num=1234

In [17]:
datetimes[num]

Timestamp('2017-05-30 02:35:00')

In [18]:
pred=yhat[num,:,0].detach().cpu()
pred=pred*std+mean
pred=pred.tolist()
pred

[75.15680694580078,
 48.70212936401367,
 63.353309631347656,
 62.55005645751953,
 70.97206115722656,
 62.942543029785156,
 65.8260269165039,
 62.93246841430664,
 56.31621170043945,
 64.26952362060547,
 59.64177322387695,
 67.01722717285156,
 67.06369018554688,
 70.57037353515625,
 64.8479232788086,
 66.40425872802734,
 61.50917053222656,
 63.11659622192383,
 66.73084259033203,
 70.68212890625,
 57.807315826416016,
 46.77021408081055,
 67.78660583496094,
 64.24658203125,
 67.74715423583984,
 70.57945251464844,
 63.3056755065918,
 66.72669219970703,
 63.540958404541016,
 66.55863952636719,
 64.00801849365234,
 58.200557708740234,
 71.7557601928711,
 69.84725952148438,
 66.260009765625,
 61.82664489746094,
 75.25335693359375,
 68.20204162597656,
 68.8822021484375,
 65.2734146118164,
 67.8205337524414,
 58.04322052001953,
 58.042625427246094,
 62.03352355957031,
 62.271217346191406,
 66.42676544189453,
 61.87931823730469,
 68.5284652709961,
 66.12413787841797,
 63.02887725830078,
 54.40715

In [19]:
import folium
 
 
m=folium.Map(location=(mean_lat,mean_lon),zoom_start=12)
 
for i in range(len(df)):
    #print(i)
    tmp_lat=df.at[i,'lat']
    tmp_lon=df.at[i,'lon']
    tmp_id=df.at[i,'station_id']
    #print(tmp_lat,tmp_lon)
    folium.Marker(location=(tmp_lat,tmp_lon),
                  icon=folium.Icon(icon='car',
                                   prefix='fa',
                                  color='green',
                                  icon_color='blue'),
                 popup='id: '+str(int(tmp_id))+' speed: '+str(pred[i])).add_to(m)
m

# 5 predict approximate turning prob (terminated）

Using station's observation speed t o approximate turning probability is a quite new concept. Few computer science paper has discussed this topic. As turning probability is a quite tough and complicated thing, we use the following assumption:

- Speed Differential as a Traffic Flow Indicator:

In traffic engineering, speed is one of the key indicators for measuring road performance and traffic volume. Typically, higher speeds indicate lower traffic density, while lower speeds may signify higher traffic density. Speed differentials, such as $V_2-V_1$, can be viewed as indicators of changes in traffic conditions. A larger speed differential might imply a relatively lower traffic flow in a certain direction.

- Emphasizing Speed Differences Using Reciprocals:

Adopting the reciprocal of speed differentials ($\frac{1}{V_2-V_1}$) is a method to emphasize the importance of speed variances. In this concept, a smaller speed differential (indicating higher traffic density) results in a larger reciprocal value, thereby assigning greater weight to that direction in calculations.

In the absence of detailed traffic flow data, this approach attempts to extract valuable insights from limited information (i.e., speed data). It is a simplified yet potentially effective method for estimating the turning probabilities for different road sections.

We define $V_1,V_2,V_3,V_4$ and speed for 4 stations of one crossing.

Each direction's speed gap for station 1 is $\Delta V_i=V_{i+1}-V_1,i \in \{1,2,3\}$

The corresponding reciprocal of speed gap is $\Delta rV_i=\frac{1}{V_{i+1}-V_1},i \in \{1,2,3\}$

Then we use softmax to approximate each turning direction's probability of station 1:

- $P_{1 \rightarrow 2}=\frac{e^{\frac{1}{V_{2}-V_1}}}{e^{\frac{1}{V_{2}-V_1}}+e^{\frac{1}{V_{3}-V_1}}+e^{\frac{1}{V_{4}-V_1}}}$
- $P_{1 \rightarrow 3}=\frac{e^{\frac{1}{V_{3}-V_1}}}{e^{\frac{1}{V_{2}-V_1}}+e^{\frac{1}{V_{3}-V_1}}+e^{\frac{1}{V_{4}-V_1}}}$
- $P_{1 \rightarrow 4}=\frac{e^{\frac{1}{V_{4}-V_1}}}{e^{\frac{1}{V_{2}-V_1}}+e^{\frac{1}{V_{3}-V_1}}+e^{\frac{1}{V_{4}-V_1}}}$

that is

$softmax(\frac{1}{|v_2-v_1|},\frac{1}{|v_3-v_1|},\frac{1}{|v_4-v_1|})$

In [20]:
import torch

In [21]:
tst=torch.Tensor([1/(63.21-60.17),1/(64.35-60.17),1/(67.03-60.17)])
tst

tensor([0.3289, 0.2392, 0.1458])

In [22]:
torch.softmax(tst,dim=0)

tensor([0.3641, 0.3328, 0.3031])

# 6 更新版：如何计算转向概率

## 6.1 速度差

**速度差**指的是驶出路段和驶入路段之间的速度差异。

影响：

- **速度差正值较大**（即驶入路段速度明显高于驶出路段）：这通常意味着驶入路段比驶出路段拥堵程度低，流通性更好。在这种情况下，司机可能更倾向于转向这个流畅的路段，因此**转向概率增加**。

- **速度差负值较大**（即驶入路段速度低于驶出路段）：这表明驶入路段比驶出路段更拥堵。因此，司机可能会避免转入这个路段，**转向概率减少**。

## 6.2 驶入路段速度 

**驶入路段速度**指的是潜在转向路段的速度。

影响：
- **驶入路段速度较高**：这通常表示该路段流通性好，拥堵程度低。司机可能倾向于转向这样的路段，以避免拥堵，因此**转向这个路段的概率增加**。
- **驶入路段速度较低**：如果潜在的转向路段速度低，表明该路段可能较为拥堵。在这种情况下，司机可能会避免转入这个路段，**转向概率减少**。

所以更新后的 吸引函数为： （废弃）

$P=\frac{1}{1+e^{-(a \cdot \Delta V+b \cdot V_{in}-c \cdot V_{out})}}$

- $P$ 是从驶出路段转向驶入路段的概率
- $\Delta V=V_{in}-V_{out}$ 是速度差
- $V_{in}$是驶入路段（经过路口之后）的速度
- $V_{out}$是输出路段（经过路口之前）的速度

所以更新后的 吸引函数为：

$P=\frac{1}{1+e^{-(a \cdot \Delta V+b \cdot \sqrt{V_{in})}}}$

- $P$ 是从驶出路段转向驶入路段的概率
- $\Delta V=V_{in}-V_{out}$ 是速度差
- $V_{in}$是驶入路段（经过路口之后）的速度

这个函数模型通过结合速度差、驶出路段速度、和驶入路段速度的影响，提供了一个综合的视角来预测司机的转向行为。它采用逻辑斯蒂形式来确保预测的转向概率始终在0到1之间，并能够捕捉到这些因素的非线性和相互作用效应。通过调整系数 a，b，c，可以根据具体的交通环境和司机行为模式来调整模型。

同样地，通过这种方式得到某一驶出路段的各个驶入路段的吸引值 P1 P2 P3，然后我们使用softmax(P1,P2,P3)得到对应的概率

### 优势 - 效果方面

- 捕捉复杂交通动态：通过融合速度差、驶出路段速度、和驶入路段速度，这种方法能够更全面地理解和预测司机的行为，尤其是在复杂的交通环境中。
- 灵活适应多变交通状况：参数化的模型允许根据不同的交通环境和司机行为模式进行调整，提供高度的适应性。
- 更好的预测准确性：逻辑斯蒂形式的函数确保了预测值在合理的范围内（即0到1之间），避免了不现实的概率估计。
- 非线性特性的体现：这种模型能够表现出交通流中的非线性特性，如在高速路段小的速度变化可能不会显著影响司机行为，而在低速路段则可能有显著影响。

# 优势 - 效率方面
- 计算效率：虽然模型包含非线性元素，但其计算依然可以保持高效，特别是当使用现代计算工具和优化算法时。
- 易于实现和集成：这种模型的结构相对简单，可以容易地在各种交通模拟软件中实现和集成。
- 适用于实时或近实时分析：由于计算效率高，这种方法适合用于实时或近实时的交通流分析，有助于交通管理和控制。

# 参数a，b，c的考量

## a
在高速公路上，驶入路段的速度如果明显高于驶出路段，这对驾驶者来说是一个强烈的吸引因素，因为在高速行驶条件下，即使是小的速度提升也可以显著减少旅行时间。

所以a相对较大，以反映速度差在高速公路上对转向决策的重要性

## b
考虑到高速公路上速度的变化范围相对较小，b 的值可能不需要像 a 那么高

## c
c 的值也应该相对小一些，以考虑到即使在高速条件下，驾驶者可能仍然会为了更高的速度或更好的路况做出转向决策

![](example.png)

In [23]:
a=1
b=0.05
def func_prob(v_in,v_out):
    #v_in is a float num, v_out is a tensor
    v_out_expanded = v_out.unsqueeze(1).expand_as(v_in)
    delta_v=v_in-v_out_expanded 
    #print(a*delta_v,b* torch.sqrt(v_in))
    prob=1/(1+torch.exp(-(a*delta_v+b* torch.sqrt(v_in))))
    #print(prob)
    #print(a*delta_v,c*v_in_expanded)
    return torch.softmax(prob,dim=1)

v_out=torch.tensor([65.64])
v_in=torch.tensor([[61.40,64.75,63.9]])
#直行 左转 右转
func_prob(v_in,v_out)

tensor([[0.2749, 0.3938, 0.3313]])

In [24]:
pred_out=yhat[:,173,0].detach().cpu()
pred_out=pred_out*std+mean
pred_out

tensor([54.9982, 58.2261, 60.7750,  ..., 68.3972, 68.1306, 68.9906])

In [25]:
pred_straight=yhat[:,252,0].detach().cpu()
pred_left=yhat[:,239,0].detach().cpu()
pred_right=yhat[:,290,0].detach().cpu()
pred_straight=pred_straight*std+mean
pred_left=pred_left*std+mean
pred_right=pred_right*std+mean
pred_straight[:5]

tensor([54.9650, 56.6980, 59.2391, 60.3084, 60.9604])

In [26]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [401846,407331,404640,404435]):
            print(tmp_id,i)

401846 173
404435 239
404640 252
407331 290


In [27]:
torch.stack((pred_straight,pred_left,pred_right),dim=1)

tensor([[54.9650, 65.4350, 64.1662],
        [56.6980, 65.0561, 64.4230],
        [59.2391, 65.4447, 64.7144],
        ...,
        [62.7354, 64.9006, 68.4989],
        [62.6457, 64.5205, 68.9160],
        [63.1983, 64.6835, 68.4141]])

In [28]:
res_tensor_prob=func_prob(torch.stack((pred_straight,pred_left,pred_right),dim=1),pred_out)
res_tensor_prob

tensor([[0.2480, 0.3760, 0.3760],
        [0.1897, 0.4053, 0.4050],
        [0.1910, 0.4058, 0.4031],
        ...,
        [0.2564, 0.2664, 0.4772],
        [0.2394, 0.2474, 0.5132],
        [0.2785, 0.2827, 0.4388]])

In [29]:
turning_prob=pd.DataFrame(res_tensor_prob,
                          index=datetimes,
                         columns=['straight', 'left', 'right'])
turning_prob

,straight,left,right
2017-05-25 19:45:00,0.247959,0.376030,0.376011
2017-05-25 19:50:00,0.189708,0.405275,0.405016
2017-05-25 19:55:00,0.191033,0.405825,0.403142
2017-05-25 20:00:00,0.187058,0.421959,0.390983
2017-05-25 20:05:00,0.229593,0.396407,0.374000
...,...,...,...
2017-06-30 23:35:00,0.250255,0.273507,0.476237
2017-06-30 23:40:00,0.219824,0.256848,0.523328
2017-06-30 23:45:00,0.256429,0.266421,0.477150
2017-06-30 23:50:00,0.239415,0.247379,0.513206


In [30]:
turning_prob.describe()

,straight,left,right
count,10419.000000,10419.000000,10419.000000
mean,0.296001,0.331402,0.372596
std,0.047409,0.051541,0.066718
min,0.155420,0.212194,0.165724
25%,0.264836,0.307419,0.332140
50%,0.311670,0.329896,0.346584
75%,0.331633,0.342712,0.401975
max,0.524025,0.551013,0.575668


![](pemsbay-1.png)

In [31]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [401507,401937,400869,400174,401936,401534]):
            print(tmp_id,i)

400174 27
400869 119
401507 156
401534 157
401936 179
401937 180


## for 401534

In [37]:
pred_straight=yhat[:,156,0].detach().cpu()
#773939

pred_left=yhat[:,119,0].detach().cpu()

pred_out=yhat[:,157,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight','left'])
f='METRLA_res/'+'401534'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,left
0,2017-05-25 19:45:00,0.728406,0.271593
1,2017-05-25 19:50:00,0.729555,0.270445
2,2017-05-25 19:55:00,0.728623,0.271377
3,2017-05-25 20:00:00,0.721640,0.278360
4,2017-05-25 20:05:00,0.721796,0.278204
...,...,...,...
10414,2017-06-30 23:35:00,0.649182,0.350818
10415,2017-06-30 23:40:00,0.657009,0.342991
10416,2017-06-30 23:45:00,0.596698,0.403302
10417,2017-06-30 23:50:00,0.605713,0.394287


## for 400174

In [36]:

pred_right=yhat[:,179,0].detach().cpu()
#7680
pred_left=yhat[:,156,0].detach().cpu()

pred_out=yhat[:,27,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['right','left'])
f='METRLA_res/'+'400174'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,right,left
0,2017-05-25 19:45:00,0.357556,0.642444
1,2017-05-25 19:50:00,0.300449,0.699551
2,2017-05-25 19:55:00,0.284957,0.715043
3,2017-05-25 20:00:00,0.298772,0.701228
4,2017-05-25 20:05:00,0.298615,0.701385
...,...,...,...
10414,2017-06-30 23:35:00,0.459069,0.540931
10415,2017-06-30 23:40:00,0.471790,0.528210
10416,2017-06-30 23:45:00,0.476422,0.523578
10417,2017-06-30 23:50:00,0.502116,0.497885


## for 401937

In [34]:
pred_straight=yhat[:,179,0].detach().cpu()
#773939
pred_right=yhat[:,119,0].detach().cpu()


pred_out=yhat[:,180,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right'])
f='METRLA_res/'+'401937'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right
0,2017-05-25 19:45:00,0.534032,0.465968
1,2017-05-25 19:50:00,0.652845,0.347155
2,2017-05-25 19:55:00,0.548105,0.451895
3,2017-05-25 20:00:00,0.557002,0.442998
4,2017-05-25 20:05:00,0.497803,0.502197
...,...,...,...
10414,2017-06-30 23:35:00,0.581876,0.418124
10415,2017-06-30 23:40:00,0.572498,0.427502
10416,2017-06-30 23:45:00,0.566361,0.433638
10417,2017-06-30 23:50:00,0.624118,0.375882


![](pemsbay-2.png)

In [38]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [401845,401846,404435,404640,407331,407373]):
            print(tmp_id,i)

401845 172
401846 173
404435 239
404640 252
407331 290
407373 308


## for 407373

In [42]:
pred_straight=yhat[:,239,0].detach().cpu()
#773939
pred_right=yhat[:,172,0].detach().cpu()
#7680
pred_left=yhat[:,252,0].detach().cpu()

pred_out=yhat[:,308,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'404640'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.333334,0.333334,0.333331
1,2017-05-25 19:50:00,0.333334,0.333334,0.333333
2,2017-05-25 19:55:00,0.333334,0.333334,0.333333
3,2017-05-25 20:00:00,0.333333,0.333333,0.333333
4,2017-05-25 20:05:00,0.333333,0.333333,0.333333
...,...,...,...,...
10414,2017-06-30 23:35:00,0.334041,0.334138,0.331821
10415,2017-06-30 23:40:00,0.339025,0.339933,0.321042
10416,2017-06-30 23:45:00,0.334546,0.335095,0.330359
10417,2017-06-30 23:50:00,0.333461,0.333545,0.332994


## for 404640

In [41]:
pred_straight=yhat[:,172,0].detach().cpu()
#773939
pred_right=yhat[:,290,0].detach().cpu()
#7680
pred_left=yhat[:,239,0].detach().cpu()

pred_out=yhat[:,252,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'404640'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.333342,0.333321,0.333337
1,2017-05-25 19:50:00,0.333382,0.333286,0.333332
2,2017-05-25 19:55:00,0.333780,0.332869,0.333351
3,2017-05-25 20:00:00,0.334784,0.332044,0.333171
4,2017-05-25 20:05:00,0.335408,0.331919,0.332673
...,...,...,...,...
10414,2017-06-30 23:35:00,0.336454,0.336118,0.327429
10415,2017-06-30 23:40:00,0.336620,0.336363,0.327017
10416,2017-06-30 23:45:00,0.341366,0.340699,0.317935
10417,2017-06-30 23:50:00,0.343535,0.343295,0.313170


## for 404435

In [40]:
pred_straight=yhat[:,290,0].detach().cpu()
#773939
pred_right=yhat[:,252,0].detach().cpu()
#7680
pred_left=yhat[:,172,0].detach().cpu()

pred_out=yhat[:,239,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'404435'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.277092,0.206189,0.516720
1,2017-05-25 19:50:00,0.299776,0.192686,0.507538
2,2017-05-25 19:55:00,0.294408,0.194267,0.511325
3,2017-05-25 20:00:00,0.304577,0.192200,0.503223
4,2017-05-25 20:05:00,0.321041,0.190229,0.488729
...,...,...,...,...
10414,2017-06-30 23:35:00,0.411980,0.166779,0.421242
10415,2017-06-30 23:40:00,0.413975,0.165069,0.420956
10416,2017-06-30 23:45:00,0.408149,0.176805,0.415046
10417,2017-06-30 23:50:00,0.407997,0.182163,0.409841


## for 401846

In [39]:
pred_straight=yhat[:,252,0].detach().cpu()
#773939
pred_right=yhat[:,239,0].detach().cpu()
#7680
pred_left=yhat[:,290,0].detach().cpu()

pred_out=yhat[:,173,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'401846'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.247959,0.376030,0.376011
1,2017-05-25 19:50:00,0.189708,0.405275,0.405016
2,2017-05-25 19:55:00,0.191033,0.405825,0.403142
3,2017-05-25 20:00:00,0.187058,0.421959,0.390983
4,2017-05-25 20:05:00,0.229593,0.396407,0.374001
...,...,...,...,...
10414,2017-06-30 23:35:00,0.250255,0.273507,0.476237
10415,2017-06-30 23:40:00,0.219824,0.256848,0.523328
10416,2017-06-30 23:45:00,0.256429,0.266421,0.477150
10417,2017-06-30 23:50:00,0.239415,0.247379,0.513206


![](pemsbay-3.png)

In [43]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [400464,400907,400832,400649,400934,400485,400240,400464]):
            print(tmp_id,i)

400240 39
400464 67
400485 69
400649 86
400832 115
400907 123
400934 127


## for 400464

In [47]:
pred_straight=yhat[:,115,0].detach().cpu()
#773939
pred_right=yhat[:,67,0].detach().cpu()
#7680
pred_left=yhat[:,127,0].detach().cpu()

pred_out=yhat[:,67,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400464'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.406560,0.382714,0.210725
1,2017-05-25 19:50:00,0.468912,0.342472,0.188616
2,2017-05-25 19:55:00,0.436990,0.363135,0.199875
3,2017-05-25 20:00:00,0.422517,0.372505,0.204978
4,2017-05-25 20:05:00,0.449960,0.354754,0.195287
...,...,...,...,...
10414,2017-06-30 23:35:00,0.231887,0.399551,0.368562
10415,2017-06-30 23:40:00,0.233086,0.415322,0.351591
10416,2017-06-30 23:45:00,0.248582,0.448754,0.302665
10417,2017-06-30 23:50:00,0.218590,0.395023,0.386387


## for 400485

In [46]:
pred_straight=yhat[:,67,0].detach().cpu()
#773939
pred_right=yhat[:,39,0].detach().cpu()
#7680
pred_left=yhat[:,115,0].detach().cpu()

pred_out=yhat[:,69,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400485'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.252295,0.480467,0.267238
1,2017-05-25 19:50:00,0.211929,0.473375,0.314696
2,2017-05-25 19:55:00,0.237830,0.477853,0.284316
3,2017-05-25 20:00:00,0.236231,0.507083,0.256686
4,2017-05-25 20:05:00,0.221397,0.522356,0.256247
...,...,...,...,...
10414,2017-06-30 23:35:00,0.247961,0.545409,0.206629
10415,2017-06-30 23:40:00,0.266576,0.532435,0.200988
10416,2017-06-30 23:45:00,0.267339,0.529636,0.203025
10417,2017-06-30 23:50:00,0.277959,0.522960,0.199082


## for 400649

In [45]:
pred_straight=yhat[:,39,0].detach().cpu()
#773939
pred_right=yhat[:,127,0].detach().cpu()
#7680
pred_left=yhat[:,67,0].detach().cpu()

pred_out=yhat[:,86,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400649'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.495752,0.245598,0.258650
1,2017-05-25 19:50:00,0.518883,0.236717,0.244399
2,2017-05-25 19:55:00,0.494856,0.244642,0.260503
3,2017-05-25 20:00:00,0.531740,0.221352,0.246908
4,2017-05-25 20:05:00,0.544094,0.222734,0.233172
...,...,...,...,...
10414,2017-06-30 23:35:00,0.543884,0.225045,0.231071
10415,2017-06-30 23:40:00,0.527067,0.225618,0.247315
10416,2017-06-30 23:45:00,0.532110,0.216931,0.250959
10417,2017-06-30 23:50:00,0.516809,0.240093,0.243099


## for 400907

In [44]:
pred_straight=yhat[:,127,0].detach().cpu()
#773939
pred_right=yhat[:,115,0].detach().cpu()
#7680
pred_left=yhat[:,39,0].detach().cpu()

pred_out=yhat[:,123,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400907'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.155362,0.422319,0.422319
1,2017-05-25 19:50:00,0.155362,0.422319,0.422319
2,2017-05-25 19:55:00,0.155364,0.422313,0.422323
3,2017-05-25 20:00:00,0.155499,0.421838,0.422663
4,2017-05-25 20:05:00,0.155665,0.421232,0.423103
...,...,...,...,...
10414,2017-06-30 23:35:00,0.228637,0.216225,0.555138
10415,2017-06-30 23:40:00,0.230693,0.215333,0.553974
10416,2017-06-30 23:45:00,0.229996,0.225630,0.544374
10417,2017-06-30 23:50:00,0.254137,0.244848,0.501015


![](pemsbay-4.png)

In [48]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [401400,401388,400100,404370,401163,401167,400951,408911]):
            print(tmp_id,i)

400100 16
400951 128
401163 141
401167 142
401388 147
401400 149
404370 237
408911 313


## for 408911

In [52]:
pred_straight=yhat[:,16,0].detach().cpu()
#773939
pred_right=yhat[:,149,0].detach().cpu()
#7680
pred_left=yhat[:,141,0].detach().cpu()

pred_out=yhat[:,313,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'408911'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.379644,0.315731,0.304625
1,2017-05-25 19:50:00,0.467452,0.355431,0.177117
2,2017-05-25 19:55:00,0.452822,0.378646,0.168532
3,2017-05-25 20:00:00,0.512285,0.273018,0.214697
4,2017-05-25 20:05:00,0.506876,0.261272,0.231852
...,...,...,...,...
10414,2017-06-30 23:35:00,0.304326,0.319436,0.376238
10415,2017-06-30 23:40:00,0.251744,0.370093,0.378163
10416,2017-06-30 23:45:00,0.241590,0.366989,0.391421
10417,2017-06-30 23:50:00,0.312455,0.340603,0.346942


## for 401167

In [51]:
pred_straight=yhat[:,149,0].detach().cpu()
#773939
pred_right=yhat[:,128,0].detach().cpu()
#7680
pred_left=yhat[:,16,0].detach().cpu()

pred_out=yhat[:,142,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'401167'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.217568,0.563329,0.219103
1,2017-05-25 19:50:00,0.213074,0.570654,0.216272
2,2017-05-25 19:55:00,0.216750,0.562466,0.220784
3,2017-05-25 20:00:00,0.227901,0.540757,0.231342
4,2017-05-25 20:05:00,0.258289,0.479483,0.262229
...,...,...,...,...
10414,2017-06-30 23:35:00,0.334979,0.331124,0.333897
10415,2017-06-30 23:40:00,0.343827,0.329016,0.327157
10416,2017-06-30 23:45:00,0.336883,0.332058,0.331059
10417,2017-06-30 23:50:00,0.341690,0.332873,0.325437


## for 404370

In [50]:
pred_straight=yhat[:,128,0].detach().cpu()
#773939
pred_right=yhat[:,141,0].detach().cpu()
#7680
pred_left=yhat[:,149,0].detach().cpu()

pred_out=yhat[:,237,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'404370'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.572593,0.213243,0.214164
1,2017-05-25 19:50:00,0.574486,0.212433,0.213081
2,2017-05-25 19:55:00,0.567520,0.216130,0.216351
3,2017-05-25 20:00:00,0.568139,0.215715,0.216146
4,2017-05-25 20:05:00,0.537739,0.230911,0.231350
...,...,...,...,...
10414,2017-06-30 23:35:00,0.211771,0.528141,0.260089
10415,2017-06-30 23:40:00,0.215025,0.451407,0.333568
10416,2017-06-30 23:45:00,0.215823,0.508261,0.275915
10417,2017-06-30 23:50:00,0.233439,0.502378,0.264183


## for 401388

In [49]:
pred_straight=yhat[:,141,0].detach().cpu()
#773939
pred_right=yhat[:,16,0].detach().cpu()
#7680
pred_left=yhat[:,128,0].detach().cpu()

pred_out=yhat[:,147,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'401388'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.155363,0.422318,0.422318
1,2017-05-25 19:50:00,0.155389,0.422305,0.422305
2,2017-05-25 19:55:00,0.333216,0.333392,0.333392
3,2017-05-25 20:00:00,0.333333,0.333333,0.333333
4,2017-05-25 20:05:00,0.333333,0.333333,0.333333
...,...,...,...,...
10414,2017-06-30 23:35:00,0.351399,0.347520,0.301081
10415,2017-06-30 23:40:00,0.336988,0.328283,0.334729
10416,2017-06-30 23:45:00,0.338829,0.326470,0.334701
10417,2017-06-30 23:50:00,0.335982,0.329890,0.334128


![](pemsbay-5.png)

In [53]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [402364,402365,400479,400045,400922,400965,400122,404759]):
            print(tmp_id,i)

400045 4
400122 19
400479 68
400922 126
400965 132
402364 216
402365 217
404759 254


## for 404759

In [57]:
pred_straight=yhat[:,68,0].detach().cpu()
#773939
pred_right=yhat[:,216,0].detach().cpu()
#7680
pred_left=yhat[:,126,0].detach().cpu()

pred_out=yhat[:,254,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'404759'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.422239,0.422241,0.155520
1,2017-05-25 19:50:00,0.422180,0.422181,0.155639
2,2017-05-25 19:55:00,0.422291,0.422322,0.155387
3,2017-05-25 20:00:00,0.420717,0.423489,0.155793
4,2017-05-25 20:05:00,0.420227,0.423848,0.155925
...,...,...,...,...
10414,2017-06-30 23:35:00,0.502881,0.269586,0.227533
10415,2017-06-30 23:40:00,0.498696,0.282170,0.219134
10416,2017-06-30 23:45:00,0.384363,0.374440,0.241197
10417,2017-06-30 23:50:00,0.357692,0.357443,0.284865


## for 400965

In [56]:
pred_straight=yhat[:,216,0].detach().cpu()
#773939
pred_right=yhat[:,19,0].detach().cpu()
#7680
pred_left=yhat[:,68,0].detach().cpu()

pred_out=yhat[:,132,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400965'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.463427,0.268315,0.268259
1,2017-05-25 19:50:00,0.428640,0.285822,0.285539
2,2017-05-25 19:55:00,0.450147,0.275745,0.274107
3,2017-05-25 20:00:00,0.502622,0.249603,0.247775
4,2017-05-25 20:05:00,0.491460,0.255043,0.253496
...,...,...,...,...
10414,2017-06-30 23:35:00,0.268430,0.238983,0.492588
10415,2017-06-30 23:40:00,0.293444,0.301960,0.404597
10416,2017-06-30 23:45:00,0.321050,0.338502,0.340448
10417,2017-06-30 23:50:00,0.296499,0.394615,0.308886


## for 400045

In [55]:
pred_straight=yhat[:,19,0].detach().cpu()
#773939
pred_right=yhat[:,126,0].detach().cpu()
#7680
pred_left=yhat[:,216,0].detach().cpu()

pred_out=yhat[:,4,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400045'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.332332,0.179578,0.488090
1,2017-05-25 19:50:00,0.234150,0.207549,0.558301
2,2017-05-25 19:55:00,0.256127,0.202955,0.540918
3,2017-05-25 20:00:00,0.286232,0.192616,0.521152
4,2017-05-25 20:05:00,0.251320,0.203037,0.545643
...,...,...,...,...
10414,2017-06-30 23:35:00,0.334641,0.330032,0.335328
10415,2017-06-30 23:40:00,0.335192,0.330072,0.334736
10416,2017-06-30 23:45:00,0.347434,0.306588,0.345978
10417,2017-06-30 23:50:00,0.370853,0.259966,0.369181


## for 402365

In [54]:
pred_straight=yhat[:,126,0].detach().cpu()
#773939
pred_right=yhat[:,68,0].detach().cpu()
#7680
pred_left=yhat[:,19,0].detach().cpu()

pred_out=yhat[:,217,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'402365'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.333333,0.333333,0.333333
1,2017-05-25 19:50:00,0.333333,0.333333,0.333333
2,2017-05-25 19:55:00,0.333333,0.333333,0.333333
3,2017-05-25 20:00:00,0.333333,0.333333,0.333333
4,2017-05-25 20:05:00,0.333333,0.333333,0.333333
...,...,...,...,...
10414,2017-06-30 23:35:00,0.209319,0.537446,0.253235
10415,2017-06-30 23:40:00,0.203167,0.461652,0.335182
10416,2017-06-30 23:45:00,0.197820,0.403984,0.398196
10417,2017-06-30 23:50:00,0.196921,0.359668,0.443411


![](pemsbay-6.png)

In [58]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [402364,401391,401906,402361,407172,400665,401890,401891]):
            print(tmp_id,i)

400665 89
401391 148
401890 174
401891 175
401906 176
402361 213
402364 216
407172 266


## for  401891

In [62]:
pred_straight=yhat[:,176,0].detach().cpu()
#773939
pred_right=yhat[:,216,0].detach().cpu()
#7680
pred_left=yhat[:,266,0].detach().cpu()

pred_out=yhat[:,175,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'401891'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.155547,0.422226,0.422226
1,2017-05-25 19:50:00,0.157499,0.421250,0.421250
2,2017-05-25 19:55:00,0.157253,0.421373,0.421373
3,2017-05-25 20:00:00,0.155794,0.422103,0.422103
4,2017-05-25 20:05:00,0.155606,0.422199,0.422196
...,...,...,...,...
10414,2017-06-30 23:35:00,0.311446,0.314963,0.373591
10415,2017-06-30 23:40:00,0.313433,0.314248,0.372319
10416,2017-06-30 23:45:00,0.286603,0.287811,0.425586
10417,2017-06-30 23:50:00,0.306505,0.307291,0.386204


## for 400665

In [61]:
pred_straight=yhat[:,216,0].detach().cpu()
#773939
pred_right=yhat[:,174,0].detach().cpu()
#7680
pred_left=yhat[:,176,0].detach().cpu()

pred_out=yhat[:,89,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400665'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.229713,0.557888,0.212399
1,2017-05-25 19:50:00,0.236340,0.538270,0.225390
2,2017-05-25 19:55:00,0.228655,0.556591,0.214754
3,2017-05-25 20:00:00,0.240188,0.536449,0.223363
4,2017-05-25 20:05:00,0.279525,0.481224,0.239251
...,...,...,...,...
10414,2017-06-30 23:35:00,0.276726,0.465833,0.257441
10415,2017-06-30 23:40:00,0.295028,0.415830,0.289142
10416,2017-06-30 23:45:00,0.230671,0.541430,0.227899
10417,2017-06-30 23:50:00,0.233255,0.539444,0.227301


## for  402361

In [60]:
pred_straight=yhat[:,174,0].detach().cpu()
#773939
pred_right=yhat[:,266,0].detach().cpu()
#7680
pred_left=yhat[:,216,0].detach().cpu()

pred_out=yhat[:,213,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'402361'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.537850,0.220525,0.241625
1,2017-05-25 19:50:00,0.406158,0.286878,0.306964
2,2017-05-25 19:55:00,0.451468,0.302570,0.245962
3,2017-05-25 20:00:00,0.372697,0.335536,0.291766
4,2017-05-25 20:05:00,0.369925,0.311341,0.318734
...,...,...,...,...
10414,2017-06-30 23:35:00,0.336006,0.336026,0.327968
10415,2017-06-30 23:40:00,0.347642,0.349111,0.303247
10416,2017-06-30 23:45:00,0.346179,0.345956,0.307864
10417,2017-06-30 23:50:00,0.346339,0.346098,0.307563


## for 401391

In [59]:
pred_straight=yhat[:,266,0].detach().cpu()
#773939
pred_right=yhat[:,176,0].detach().cpu()
#7680
pred_left=yhat[:,174,0].detach().cpu()

pred_out=yhat[:,148,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'401391'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.331012,0.180036,0.488952
1,2017-05-25 19:50:00,0.320735,0.183249,0.496016
2,2017-05-25 19:55:00,0.334164,0.179691,0.486146
3,2017-05-25 20:00:00,0.365015,0.171997,0.462988
4,2017-05-25 20:05:00,0.343661,0.181288,0.475051
...,...,...,...,...
10414,2017-06-30 23:35:00,0.412496,0.177314,0.410190
10415,2017-06-30 23:40:00,0.437731,0.178094,0.384175
10416,2017-06-30 23:45:00,0.409050,0.162032,0.428917
10417,2017-06-30 23:50:00,0.409881,0.164821,0.425298


![](pemsbay-7.png)

In [63]:
for i in range(len(df)):
    #print(i)
    tmp_id=df.at[i,'station_id']
    if(tmp_id in [402364,403412,400065,400916,401811,401810,400563,400236]):
            print(tmp_id,i)

400065 8
400236 37
400563 78
400916 125
401810 168
401811 169
402364 216
403412 234


## for 400236

In [66]:
pred_straight=yhat[:,8,0].detach().cpu()
#773939
pred_right=yhat[:,216,0].detach().cpu()
#7680
pred_left=yhat[:,169,0].detach().cpu()

pred_out=yhat[:,37,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400236'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.211942,0.576117,0.211942
1,2017-05-25 19:50:00,0.211943,0.576116,0.211941
2,2017-05-25 19:55:00,0.212483,0.575721,0.211796
3,2017-05-25 20:00:00,0.212291,0.575858,0.211851
4,2017-05-25 20:05:00,0.227270,0.564904,0.207826
...,...,...,...,...
10414,2017-06-30 23:35:00,0.208063,0.247416,0.544521
10415,2017-06-30 23:40:00,0.215572,0.225179,0.559249
10416,2017-06-30 23:45:00,0.211854,0.221657,0.566489
10417,2017-06-30 23:50:00,0.213773,0.222014,0.564213


## for 401810

In [ ]:
pred_straight=yhat[:,216,0].detach().cpu()
#773939
pred_right=yhat[:,78,0].detach().cpu()
#7680
pred_left=yhat[:,8,0].detach().cpu()

pred_out=yhat[:,168,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'401810'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

## for 400916

In [65]:
pred_straight=yhat[:,78,0].detach().cpu()
#773939
pred_right=yhat[:,169,0].detach().cpu()
#7680
pred_left=yhat[:,216,0].detach().cpu()

pred_out=yhat[:,125,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'400916'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.423097,0.155835,0.421068
1,2017-05-25 19:50:00,0.428643,0.158318,0.413039
2,2017-05-25 19:55:00,0.436090,0.162122,0.401789
3,2017-05-25 20:00:00,0.444200,0.165107,0.390692
4,2017-05-25 20:05:00,0.425287,0.157133,0.417579
...,...,...,...,...
10414,2017-06-30 23:35:00,0.336983,0.337792,0.325226
10415,2017-06-30 23:40:00,0.350036,0.351154,0.298810
10416,2017-06-30 23:45:00,0.351795,0.352505,0.295700
10417,2017-06-30 23:50:00,0.349143,0.349895,0.300962


## for 403412

In [64]:
pred_straight=yhat[:,169,0].detach().cpu()
#773939
pred_right=yhat[:,8,0].detach().cpu()
#7680
pred_left=yhat[:,78,0].detach().cpu()

pred_out=yhat[:,234,0].detach().cpu()
#717492


pred_out=pred_out*std+mean
pred_straight=pred_straight*std+mean
pred_right=pred_right*std+mean
pred_left=pred_left*std+mean

res_tensor_prob=func_prob(torch.stack((pred_straight,pred_right,pred_left),dim=1),pred_out)

turning_prob = pd.DataFrame(res_tensor_prob,
                            index=datetimes[-10419:],
                            columns=['straight', 'right','left'])
f='METRLA_res/'+'403412'+'pemsbay.csv'
turning_prob.to_csv(f)
pd.read_csv(f)

,Unnamed: 0,straight,right,left
0,2017-05-25 19:45:00,0.211942,0.211942,0.576117
1,2017-05-25 19:50:00,0.211923,0.212010,0.576067
2,2017-05-25 19:55:00,0.211847,0.212292,0.575861
3,2017-05-25 20:00:00,0.211851,0.212280,0.575869
4,2017-05-25 20:05:00,0.210355,0.217856,0.571790
...,...,...,...,...
10414,2017-06-30 23:35:00,0.427682,0.163876,0.408442
10415,2017-06-30 23:40:00,0.334155,0.331699,0.334146
10416,2017-06-30 23:45:00,0.344614,0.310802,0.344584
10417,2017-06-30 23:50:00,0.361500,0.277415,0.361085
